In [3]:
from utils.tf import *
from utils.db import *
from utils.gd import * 
from utils.clt_repetidos import *
from datetime import datetime
import cx_Oracle
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go
from utils.clt_repetidos import * 
from ydata_profiling import ProfileReport
import dask.dataframe as dd
import gc
import matplotlib.pyplot as plt

# Get the data and create the output directory (Extraction)

In [4]:
path_dir_data = "/home/ale1726/proyects/datalake/clientes/data/productos/SIPE/data"

In [5]:
# Obtenemos la fecha actual para crear un directorio
# donde se guardaran los archivos de salida
date_now = datetime.now().strftime("%d_%m_%Y") 
path_dir_today =  os.path.join(path_dir_data, date_now)

# Creamos el directorio si no existe
os.makedirs(path_dir_today, exist_ok=True)

In [6]:
query_productos_sipe = """ 
WITH CLIENTES_ACTIVOS AS (
    SELECT  BCO_CLAVE_FFON, BCO_CLAVE, BCO_NOMBRE, SPB.SUC_NUMERO
    FROM SIPE.SIPE_BANCOS sp
  	LEFT JOIN SIPE.SIPE_SUCURSALES_BANCO SPB ON SP.BCO_CLAVE = SPB.SUC_BCO_CLAVE
    WHERE SP.BCO_ESTATUS = 'A'
), CLI_CUENTAS_CONTRAPARTE AS (
	SELECT  CA.*, SCC.CIP_DIV_CLAVE, SCC.CIP_TES_CLAVE, SCC.CIP_SUC_NUMERO
	FROM CLIENTES_ACTIVOS CA
	LEFT JOIN SIPE.SIPE_CUENTAS_CONTRAPARTE SCC ON CA.SUC_NUMERO = SCC.CIP_SUC_NUMERO
), CLI_INV_PRES AS (
	SELECT CCC.*, SIP.*
	FROM CLI_CUENTAS_CONTRAPARTE CCC
	LEFT JOIN SIPE.SIPE_INVERSIONES_PRESTAMOS SIP ON ( SIP.IPE_SUC_NUMERO = CCC.CIP_SUC_NUMERO AND SIP.IPE_TES_CLAVE = CCC.CIP_TES_CLAVE AND SIP.IPE_DIV_CLAVE = CCC.CIP_DIV_CLAVE )
) SELECT * FROM CLI_INV_PRES
"""

In [7]:
path_dir_productos = get_table(path_dir_today, db_sipe, "productos_clientes_sipe", query = query_productos_sipe)

# Transform the data and output the transformation

In [8]:
path_dir_output_transformation = os.path.join("/home/ale1726/proyects/datalake/clientes/data/productos/SIPE/agrupados", date_now)
os.makedirs(path_dir_output_transformation, exist_ok=True)
path_dir_output_transformation

'/home/ale1726/proyects/datalake/clientes/data/productos/SIPE/agrupados/22_04_2025'

In [38]:
productos_sipe =  pd.read_csv(path_dir_productos, low_memory=False)

In [17]:
productos_sipe.columns

Index(['BCO_CLAVE_FFON', 'BCO_CLAVE', 'BCO_NOMBRE', 'SUC_NUMERO',
       'CIP_DIV_CLAVE', 'CIP_TES_CLAVE', 'CIP_SUC_NUMERO', 'IPE_FOLIO',
       'IPE_FECHA_REGISTRO', 'IPE_FECHA_MODIF', 'IPE_DIV_CLAVE', 'IPE_TIPO',
       'IPE_USR_CLAVE_REGISTRO', 'IPE_USR_CLAVE_MODIF', 'IPE_TST_CLAVE',
       'IPE_TIN_CLAVE', 'IPE_EMI_CLAVE', 'IPE_MIS_SERIE', 'IPE_TES_CLAVE',
       'IPE_SUC_NUMERO', 'IPE_FECHA_VALOR', 'IPE_IMPORTE', 'IPE_PLAZO',
       'IPE_TASA', 'IPE_FECHA_VENCIMIENTO', 'IPE_FOLIO_ORIGEN',
       'IPE_FECHA_CONCERTACION', 'IPE_EVENTO', 'AÑO_FOPE', 'MES_FOPE',
       'MES_STR'],
      dtype='object')

In [39]:
productos_sipe["IPE_FECHA_REGISTRO"] = pd.to_datetime(productos_sipe["IPE_FECHA_REGISTRO"])
productos_sipe["AÑO_FOPE"] = productos_sipe["IPE_FECHA_REGISTRO"].dt.year.astype('Int64')
productos_sipe["MES_FOPE"] = productos_sipe["IPE_FECHA_REGISTRO"].dt.month
productos_sipe["MES_STR"] = productos_sipe["IPE_FECHA_REGISTRO"].dt.month_name()  

In [40]:
productos_sipe = productos_sipe[productos_sipe["AÑO_FOPE"] != 2099]

In [41]:
agrupado_operaciones_anuales = (
    productos_sipe.groupby(['AÑO_FOPE'], as_index=False)
    .agg(
        num_operaciones=('IPE_FOLIO', 'count'), 
        monto_anuales_sum=('IPE_IMPORTE', 'sum'), 
        monto_anuales_min=('IPE_IMPORTE', 'min'), 
        monto_anuales_max=('IPE_IMPORTE', 'max'), 
        monto_anuales_mean=('IPE_IMPORTE', 'mean'), 
        monto_anuales_median=('IPE_IMPORTE', 'median')
    ).sort_values(by=['AÑO_FOPE'], ascending=True)
)

In [42]:
agrupado_operaciones_anuales.to_csv(os.path.join(path_dir_output_transformation, "agrupado_operaciones_anuales.csv"), index=False)

In [43]:
agrupado_operaciones_anuales_mes = (
    productos_sipe.groupby(['AÑO_FOPE','MES_FOPE', 'MES_STR'], as_index=False)
    .agg(
        num_operaciones=('IPE_FOLIO', 'count'), 
        monto_anuales_sum=('IPE_IMPORTE', 'sum'), 
        monto_anuales_min=('IPE_IMPORTE', 'min'), 
        monto_anuales_max=('IPE_IMPORTE', 'max'), 
        monto_anuales_mean=('IPE_IMPORTE', 'mean'), 
        monto_anuales_median=('IPE_IMPORTE', 'median')
    ).sort_values(by=['AÑO_FOPE','MES_FOPE'], ascending=True)
)

In [44]:
agrupado_operaciones_anuales_mes.to_csv(os.path.join(path_dir_output_transformation, "agrupado_operaciones_anuales_mes.csv"), index=False)

In [45]:
agrupado_plazo_anual = (
    productos_sipe.groupby(['AÑO_FOPE'], as_index=False)
    .agg(
        plazo_mean=('IPE_PLAZO', lambda x: round(x.mean())),
        plazo_min=('IPE_PLAZO', 'min'), 
        plazo_max=('IPE_PLAZO', 'max'),  
        plazo_median=('IPE_PLAZO', 'median')
    ).sort_values(by=['AÑO_FOPE'], ascending=True)
)

In [46]:
agrupado_plazo_anual.to_csv(os.path.join(path_dir_output_transformation, "agrupado_plazo_anual.csv"), index=False)

In [50]:
agrupado_clientes_año = (
    productos_sipe.groupby(['BCO_CLAVE','BCO_NOMBRE', 'AÑO_FOPE'], as_index=False)
    .agg(
        transacciones=('IPE_FOLIO', 'count'),
        monto_mean=('IPE_IMPORTE', lambda x: round(x.mean(), 2) if x.mean() != 0 else 0),
        plazo_promedio=('IPE_PLAZO', lambda x: f"{round(x.mean())} Días"),
        tasa_prom=('IPE_TASA', lambda x: f"{round(x.mean(),2)}%"),
    )
    .sort_values(by=['AÑO_FOPE'], ascending=False)
)

In [56]:
agrupado_clientes_año

,BCO_CLAVE,BCO_NOMBRE,AÑO_FOPE,transacciones,monto_mean,plazo_promedio,tasa_prom
346,215,THE BANK OF NOVA SCOTIA TORONTO,2025,10,5.600000e+07,10 Días,4.39%
640,344,"BARCLAYS BANK, MEXICO",2025,41,4.256098e+07,1 Días,4.22%
655,350,"MIZUHO BANK MEXICO S. A., I. B. M.",2025,44,1.609091e+07,1 Días,4.22%
581,310,"BANCO AZTECA, S.A.",2025,232,4.743739e+07,1 Días,4.22%
153,17,"MUFG BANK MEXICO, S.A.",2025,25,4.160000e+07,1 Días,4.22%
...,...,...,...,...,...,...,...
314,83,"NAFINSA HOLDING, CORP.",1997,4,6.250000e+06,2 Días,4.69%
304,82,"ROYAL BANK OF SCOTLAND, N.Y.",1997,133,1.780342e+08,2 Días,5.53%
302,81,JP MORGAN,1997,179,2.388972e+08,1 Días,5.54%
301,78,"BANCO DE ORIENTE, S.A.",1997,2,2.000000e+06,46 Días,6.75%


In [49]:
agrupado_clientes_año.to_csv(os.path.join(path_dir_output_transformation, "agrupado_clientes_año.csv"), index=False)

In [51]:
agrupado_clientes_tra_monto = (
    productos_sipe.groupby(['BCO_NOMBRE'], as_index=False)
    .agg(
        transacciones=('IPE_FOLIO', 'count'),
        montos=('IPE_IMPORTE', lambda x: round(x.mean(), 2))
    )
    .sort_values(by='BCO_NOMBRE', ascending=False)
)

In [ ]:
agrupado_clientes_tra_monto.to_csv(os.path.join(path_dir_output_transformation, "agrupado_clientes_tra_monto.csv"), index=False)

In [54]:
agrupado_clientes_tra_monto_año = (
    productos_sipe.groupby(['AÑO_FOPE','BCO_NOMBRE'], as_index=False)
    .agg(
        transacciones=('IPE_FOLIO', 'count'),
        montos=('IPE_IMPORTE', lambda x: round(x.mean(), 2))
    )
    .sort_values(by='AÑO_FOPE', ascending=False)
)

In [ ]:
agrupado_clientes_tra_monto.to_csv(os.path.join(path_dir_output_transformation, "agrupado_clientes_tra_monto.csv"), index=False)